# Consolidated Code for Clustering

In [3]:
import geopy.distance
import folium
import math
from abc import ABC, abstractmethod

In [4]:
class Point:
    def __init__(self, id, coordinates):
        self.id = id
        self.coordinates = coordinates
        self.visited = False
        self.cluster = None
        self.is_noise = False

    def distance(self, other_point):
        """
        Compute the Euclidean distance between this point and another point.
        """
        return geopy.distance.geodesic(self.coordinates, other_point.coordinates).km

    def __str__(self):
        return f'''Point {self.id}[{str(self.coordinates)}, cluster:{self.cluster}]'''

In [8]:
class ClusterFinder(ABC):
    """DBScan Clustering Object"""
    def __init__(self, dataset):
        self.dataset = dataset # List of points to do clustering
        self.cluster_count = 0 # Total number of clusters
        self.clusters = {} # Dictionary with keys(cluster_id), values (list of Points)

    @abstractmethod
    def fit(self):
        """
        Run the DBSCAN clustering algorithm.
        """
        return self.clusters

In [12]:
class DBSCANClusterFinder(ClusterFinder):
    """
    DBScan Clustering Object using DBSCAN algorithm
    input: max_gap - int maximum distance between 2 points to be considered neighbours
    min_pts: minimum number of neighbours to be considered a corepoint
    """
    def __init__(self, dataset, max_gap=1, min_pts=2):
        super().__init__(dataset)
        self.max_gap = max_gap
        self.min_pts = min_pts

    def region_query(self, point):
        """
        Returns all points within eps-distance from the given point in the dataset.
        """
        neighbors = []
        for candidate_point in self.dataset:
            if point.distance(candidate_point) < self.max_gap:
                neighbors.append(candidate_point)
        return neighbors

    def expand_cluster(self, point, neighbors):
        """
        Expands the cluster of the given point and its neighbors.
        """
        point.cluster = self.cluster_count
        self.clusters[self.cluster_count] = [point]
        i = 0
        while i < len(neighbors):
            neighbor = neighbors[i]
            if not neighbor.visited:
                neighbor.visited = True
                new_neighbors = self.region_query(neighbor)
                if len(new_neighbors) >= self.min_pts:
                    neighbors += new_neighbors
            if neighbor.cluster is None:
                neighbor.cluster = self.cluster_count
                self.clusters[self.cluster_count].append(neighbor)
            i += 1

    def fit(self):
        """
        Run the DBSCAN clustering algorithm.
        """
        for point in self.dataset:
            if not point.visited:
                point.visited = True
                neighbors = self.region_query(point)
                if len(neighbors) < self.min_pts:
                    point.is_noise = True
                else:
                    self.cluster_count += 1
                    self.expand_cluster(point, neighbors)
        return self.clusters

In [23]:
# Initialise Points
long_lats = [
    (1.31990020343696, 103.95674626542143),
    (1.3409742208234072, 103.9640548435533),
    (1.3412846014636721, 103.96400052881947),
    (1.3277199609269097, 103.95972731843452),
    (1.3173137658654994, 103.9511365061205),
    (1.3150333487355417, 103.9470373314858),
    (1.362225122765516, 103.88294192838914),
    (1.362222442765516, 103.88294193224891),
    (1.32222442765516, 103.88294193224891),
    (1.3124085243656334, 103.93869585305949),
    (1.3118890275098984, 103.93232351063924)
    ]

dataset = []
for i in range(len(long_lats)):
    dataset.append(Point(i,long_lats[i]))

In [24]:
# Running DBSCAN using OOP
dbscan = DBSCANClusterFinder(dataset, max_gap=1, min_pts=2)
output_clusters = dbscan.fit()

# Output clusters
for cluster_id, points in output_clusters.items():
    l = [point.id for point in points]
    print(f"Cluser: {cluster_id}, Cluster ID: {l}")

Cluser: 1, Cluster ID: [0, 3, 4, 5, 9, 10]
Cluser: 2, Cluster ID: [1, 2]
Cluser: 3, Cluster ID: [6, 7]


In [25]:
import folium
import geopy.distance

def visualize_clusters(clusters):
    # Create a base map
    m = folium.Map(location=[1.3409742208234072, 103.9640548435533], zoom_start=15)  # Using a sample location

    # Define cluster colors (you can expand this list for more clusters)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Add points to the map
    for cluster_id, cluster in clusters.items():
        for point in cluster:
            color = colors[(cluster_id - 1) % len(colors)]
            folium.Marker(
                location=point.coordinates,
                icon=folium.Icon(color=color),
                popup=f"ID:{point.id}, Cluster: {cluster_id}"
            ).add_to(m)

    return m

In [26]:
map_result = visualize_clusters(output_clusters)
map_result

In [41]:
# Cluster object
class DIANA_Cluster:
    def __init__(self, points):
        self.points = points

    def longest_distance(self):
        """
        Finds the maximum distance between two points in a cluster and returns the two points
        """
        max_distance = 0
        A, B = None, None
        for i in range(len(self.points)):
            for j in range(i+1, len(self.points)):
                d = self.points[i].distance(self.points[j])
                if d > max_distance:
                    max_distance = d
                    A, B = self.points[i], self.points[j]
        return max_distance, [A, B]

    def split(self, A, B):
        # Find two points with maximum dissimilarity
        cluster_a = [A]
        cluster_b = [B]

        for i in range(len(self.points)):
            if self.points[i] in (A,B):
                pass
            else:
                if self.points[i].distance(A) > self.points[i].distance(B):
                    cluster_b.append(self.points[i])
                else:
                    cluster_a.append(self.points[i])

        return DIANA_Cluster(cluster_a), DIANA_Cluster(cluster_b)

    def __str__(self):
        return f'''DIANA_Cluster:{[i.id for i in self.points]}'''


class DIANAClusterFinder(ClusterFinder):

    def __init__(self, dataset, threshold=1.0):
        super().__init__(dataset)
        self.threshold = threshold

    def fit(self):
        initial_cluster = DIANA_Cluster(self.dataset)

        output_clusters = []
        clusters_to_split = [initial_cluster]
        while len(clusters_to_split)>0:
            target_cluster = clusters_to_split.pop()
            max_distance, furthest_pts = target_cluster.longest_distance()
            if max_distance > self.threshold:
                clusters_to_split += target_cluster.split(furthest_pts[0], furthest_pts[1])
            else:
                output_clusters.append(target_cluster)

        for i in range(len(output_clusters)):
            self.clusters[i] = output_clusters[i].points

        return self.clusters

In [42]:
# Initialise Points
long_lats = [
    (1.31990020343696, 103.95674626542143),
    (1.3409742208234072, 103.9640548435533),
    (1.3412846014636721, 103.96400052881947),
    (1.3277199609269097, 103.95972731843452),
    (1.3173137658654994, 103.9511365061205),
    (1.3150333487355417, 103.9470373314858),
    (1.362225122765516, 103.88294192838914),
    (1.362222442765516, 103.88294193224891),
    (1.32222442765516, 103.88294193224891),
    (1.3124085243656334, 103.93869585305949),
    (1.3118890275098984, 103.93232351063924)
    ]

dataset = []
for i in range(len(long_lats)):
    dataset.append(Point(i,long_lats[i]))

In [43]:
# Running DBSCAN using OOP
diana = DIANAClusterFinder(dataset, threshold=1)
output_clusters = diana.fit()

# Output clusters
for cluster_id, points in output_clusters.items():
    l = [point.id for point in points]
    print(f"Cluser: {cluster_id}, Cluster ID: {l}")

Cluser: 0, Cluster ID: [8]
Cluser: 1, Cluster ID: [6, 7]
Cluser: 2, Cluster ID: [4, 5]
Cluser: 3, Cluster ID: [10, 9]
Cluser: 4, Cluster ID: [0, 3]
Cluser: 5, Cluster ID: [2, 1]


In [44]:
map_result = visualize_clusters(output_clusters)
map_result

# Implementation of DBSCAN for clustering hotspots

In [9]:
import geopy.distance
import folium
import math

In [10]:
class Point:
    def __init__(self, id, coordinates):
        self.id = id
        self.coordinates = coordinates
        self.visited = False
        self.cluster = None
        self.is_noise = False

    def distance(self, other_point):
        """
        Compute the Euclidean distance between this point and another point.
        """
        return geopy.distance.geodesic(self.coordinates, other_point.coordinates).km

    def __str__(self):
        return f'''Point {self.id}[{str(self.coordinates)}, cluster:{self.cluster}]'''

In [11]:
class DBSCAN:
    def __init__(self, dataset, eps, min_pts):
        self.dataset = dataset
        self.eps = eps
        self.min_pts = min_pts
        self.cluster_id = 0

    def region_query(self, point):
        """
        Returns all points within eps-distance from the given point in the dataset.
        """
        neighbors = []
        for candidate_point in self.dataset:
            if point.distance(candidate_point) < self.eps:
                neighbors.append(candidate_point)
        return neighbors

    def expand_cluster(self, point, neighbors):
        """
        Expands the cluster of the given point and its neighbors.
        """
        point.cluster = self.cluster_id
        i = 0
        while i < len(neighbors):
            neighbor = neighbors[i]
            if not neighbor.visited:
                neighbor.visited = True
                new_neighbors = self.region_query(neighbor)
                if len(new_neighbors) >= self.min_pts:
                    neighbors += new_neighbors
            if neighbor.cluster is None:
                neighbor.cluster = self.cluster_id
            i += 1

    def fit(self):
        """
        Run the DBSCAN clustering algorithm.
        """
        for point in self.dataset:
            if not point.visited:
                point.visited = True
                neighbors = self.region_query(point)
                if len(neighbors) < self.min_pts:
                    point.is_noise = True
                else:
                    self.cluster_id += 1
                    self.expand_cluster(point, neighbors)

In [96]:
long_lats = [
    (1.31990020343696, 103.95674626542143),
    (1.3409742208234072, 103.9640548435533),
    (1.3412846014636721, 103.96400052881947),
    (1.3277199609269097, 103.95972731843452),
    (1.3173137658654994, 103.9511365061205),
    (1.3150333487355417, 103.9470373314858),
    (1.362225122765516, 103.88294192838914),
    (1.362222442765516, 103.88294193224891),
    (1.32222442765516, 103.88294193224891),
    (1.3124085243656334, 103.93869585305949),
    (1.3118890275098984, 103.93232351063924)
    ]

dataset = []
for i in range(len(long_lats)):
    dataset.append(Point(i,long_lats[i]))

# Running DBSCAN using OOP
dbscan = DBSCAN(dataset, eps=1, min_pts=2)
dbscan.fit()

# Output clusters
for point in dataset:
    print(f"Point: {point}, Cluster ID: {point.cluster}")

Point: Point 0[(1.31990020343696, 103.95674626542143), cluster:1], Cluster ID: 1
Point: Point 1[(1.3409742208234072, 103.9640548435533), cluster:2], Cluster ID: 2
Point: Point 2[(1.3412846014636721, 103.96400052881947), cluster:2], Cluster ID: 2
Point: Point 3[(1.3277199609269097, 103.95972731843452), cluster:1], Cluster ID: 1
Point: Point 4[(1.3173137658654994, 103.9511365061205), cluster:1], Cluster ID: 1
Point: Point 5[(1.3150333487355417, 103.9470373314858), cluster:1], Cluster ID: 1
Point: Point 6[(1.362225122765516, 103.88294192838914), cluster:3], Cluster ID: 3
Point: Point 7[(1.362222442765516, 103.88294193224891), cluster:3], Cluster ID: 3
Point: Point 8[(1.32222442765516, 103.88294193224891), cluster:None], Cluster ID: None
Point: Point 9[(1.3124085243656334, 103.93869585305949), cluster:1], Cluster ID: 1
Point: Point 10[(1.3118890275098984, 103.93232351063924), cluster:1], Cluster ID: 1


In [98]:
def visualize_clusters(dataset):
    # Create a base map
    m = folium.Map(location=[1.3409742208234072, 103.9640548435533], zoom_start=15)  # Default location: NYC

    # Define cluster colors (you can expand this list for more clusters)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Add points to the map
    for point in dataset:
        cluster_id = point.cluster
        color = 'gray' if cluster_id is None else colors[(cluster_id - 1) % len(colors)]
        folium.Marker(
            location=point.coordinates,
            icon=folium.Icon(color=color),
            popup=f"ID:{point.id}, Cluster: {cluster_id}"
        ).add_to(m)

    return m

# Visualize
map_result = visualize_clusters(dataset)

In [91]:
# display(map_result)

In [92]:
dataset[3].distance(dataset[10])
print(f'{dataset[3]} and {dataset[10]} are {dataset[3].distance(dataset[10])}km apart but still the same cluster')

Point 3[(1.3277199609269097, 103.95972731843452), cluster:1] and Point 10[(1.3118890275098984, 103.93232351063924), cluster:1] are 3.516444736331772km apart but still the same cluster


## Problem: Break up large clusters

DBScan just does clustering of points as long as nodes are close together. But the overall cluster might be too large for exploration of a single drone, we need a way to limit the size of this clustering.

## DIANA

In [104]:
class DIANA_Cluster:
    def __init__(self, points):
        self.points = points

    def longest_distance(self):
        """
        Finds the maximum distance between two points in a cluster and returns the two points
        """
        max_distance = 0
        A, B = None, None
        for i in range(len(self.points)):
            for j in range(i+1, len(self.points)):
                d = self.points[i].distance(self.points[j])
                if d > max_distance:
                    max_distance = d
                    A, B = self.points[i], self.points[j]
        return max_distance, [A, B]

    def split(self, A, B):
        # Find two points with maximum dissimilarity
        cluster_a = [A]
        cluster_b = [B]

        for i in range(len(self.points)):
            if self.points[i] in (A,B):
                pass
            else:
                if self.points[i].distance(A) > self.points[i].distance(B):
                    cluster_b.append(self.points[i])
                else:
                    cluster_a.append(self.points[i])

        return DIANA_Cluster(cluster_a), DIANA_Cluster(cluster_b)

    def __str__(self):
        return f'''DIANA_Cluster:{[i.id for i in self.points]}'''


def DIANA(initial_cluster: DIANA_Cluster, threshold_km):
    output_clusters = []
    clusters_to_split = [initial_cluster]
    while len(clusters_to_split)>0:
        target_cluster = clusters_to_split.pop()
 
        max_distance, furthest_pts = target_cluster.longest_distance()
        if max_distance > threshold_km:
            clusters_to_split += target_cluster.split(furthest_pts[0], furthest_pts[1])
        else:
            output_clusters.append(target_cluster)
    return output_clusters


In [111]:
# # Example usage:

long_lats = [
    (1.31990020343696, 103.95674626542143),
    (1.3409742208234072, 103.9640548435533),
    (1.3412846014636721, 103.96400052881947),
    (1.3277199609269097, 103.95972731843452),
    (1.3173137658654994, 103.9511365061205),
    (1.3150333487355417, 103.9470373314858),
    (1.362225122765516, 103.88294192838914),
    (1.36642442765516, 103.88294193224891),
    (1.32222442765516, 103.88294193224891),
    (1.3124085243656334, 103.93869585305949),
    (1.3118890275098984, 103.93232351063924)
    ]

dataset = []
for i in range(len(long_lats)):
    dataset.append(Point(i,long_lats[i]))

initial_cluster = Cluster(dataset)

print("Initial cluster", [str(i) for i in initial_cluster.points])

clusters = DIANA(initial_cluster, threshold_km=1)

for cluster in clusters:
    print("RESULT:",[str(i) for i in cluster.points])

Initial cluster ['Point 0[(1.31990020343696, 103.95674626542143), cluster:None]', 'Point 1[(1.3409742208234072, 103.9640548435533), cluster:None]', 'Point 2[(1.3412846014636721, 103.96400052881947), cluster:None]', 'Point 3[(1.3277199609269097, 103.95972731843452), cluster:None]', 'Point 4[(1.3173137658654994, 103.9511365061205), cluster:None]', 'Point 5[(1.3150333487355417, 103.9470373314858), cluster:None]', 'Point 6[(1.362225122765516, 103.88294192838914), cluster:None]', 'Point 7[(1.36642442765516, 103.88294193224891), cluster:None]', 'Point 8[(1.32222442765516, 103.88294193224891), cluster:None]', 'Point 9[(1.3124085243656334, 103.93869585305949), cluster:None]', 'Point 10[(1.3118890275098984, 103.93232351063924), cluster:None]']
RESULT: ['Point 8[(1.32222442765516, 103.88294193224891), cluster:None]']
RESULT: ['Point 7[(1.36642442765516, 103.88294193224891), cluster:None]', 'Point 6[(1.362225122765516, 103.88294192838914), cluster:None]']
RESULT: ['Point 4[(1.3173137658654994, 10

In [112]:
import folium
import geopy.distance

def visualize_clusters2(clusters):
    # Create a base map
    m = folium.Map(location=[1.3409742208234072, 103.9640548435533], zoom_start=15)  # Using a sample location

    # Define cluster colors (you can expand this list for more clusters)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Add points to the map
    for cluster_id, cluster in enumerate(clusters, 1):
        for point in cluster.points:
            color = colors[(cluster_id - 1) % len(colors)]
            folium.Marker(
                location=point.coordinates,
                icon=folium.Icon(color=color),
                popup=f"ID:{point.id}, Cluster: {cluster_id}"
            ).add_to(m)

    return m

map_result = visualize_clusters2(clusters)
map_result
